In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('raw_data.csv')

In [3]:
# dfc = pd.read_csv('cleaned-data.csv')                               -> Can this be removed?

In [4]:
print(df.price.isna().sum())
#print(dfc.price.isna().sum())                                        -> Can this be removed?

# Dropping entries with missing values in "price" (36)
dfc = df.dropna(subset=['price'], how='all')
dfc.reset_index(drop=True, inplace=True)                              # Added "drop=True" to drop the old index

36


In [5]:
#cleaning and normalizing kitchen status
dfc.loc[dfc['equipped_kitchen'] == '0', ['equipped_kitchen']] = 'not installed'
dfc.loc[dfc['equipped_kitchen'] == 'usa uninstalled', ['equipped_kitchen']] = 'not installed'
dfc.loc[(dfc['equipped_kitchen'] != 'installed') & (dfc['equipped_kitchen'] != 'not installed'), ['equipped_kitchen']] = 'equipped'

In [6]:
#cleaning and normalizing building condition
dfc.loc[(dfc['building_condition'] == 'as new') | (dfc['building_condition'] == 'just renovated'), ['building_condition']] = 'good'
dfc.loc[dfc['building_condition'] == 'to be done up', ['building_condition']] = 'to renovate'
dfc.loc[dfc['building_condition'].isna(), ['building_condition']] = 'no info'

In [7]:
# Drop rows where 'living_area' is 0 (5 entries)
dfc = dfc[dfc["living_area"] != 0]

In [8]:
# Handling missing values in "facade_number" (9362)

# Compute median facade number by subtype
facade_dict = dfc.groupby("subtype_of_property")["facade_number"].median().to_dict()

# Impute values
dfc["facade_number"] = dfc["facade_number"].fillna(dfc["subtype_of_property"].map(facade_dict))

In [9]:
# Create function to remove 'unit' form sub type of property, so 'apartment' and 'apartment unit' become one category and so on
def remove_unit(subtype_of_property):
    if 'unit' in subtype_of_property:
        subtype_of_property = subtype_of_property.replace(' unit', '')
    return subtype_of_property

In [10]:
# Apply the function to the sub type col:
dfc['subtype_of_property'] = dfc['subtype_of_property'].apply(remove_unit)

In [11]:
dfc.to_csv('cleaned-data.csv', index=False)